In [1]:
using CSV, DataFrames, StatsBase, JuMP, Gurobi
trips = CSV.File("input_df.csv") |> DataFrame
trips_filtered = filter(row -> row[:PULocationID] in 1:10 && row[:DOLocationID] == new_time, df) 

Row,PULocationID,time_interval,DOLocationID,total_amount,duration_interval,trips,ride_percentage,next_time_interval
,Int64,Int64,Int64,Float64,Float64,Int64,Float64,Int64
1,1,0,6,118.08,1.0,2,0.5,1
2,1,0,265,93.5,1.0,2,0.5,1
3,1,1,186,161.7,1.0,1,0.5,2
4,1,1,265,59.38,1.0,1,0.5,2
5,1,2,264,71.5,1.0,1,0.5,3
6,1,2,265,19.5,1.0,1,0.5,3
7,1,3,264,81.5,1.0,1,1.0,4
8,1,5,264,156.5,1.0,1,1.0,6
9,1,6,264,67.2,1.0,1,1.0,7


In [2]:
function sample_trip(df, PULocationID, time_interval)
    
    # Filter DataFrame based on PULocationID and time_interval
    filtered_df = filter(row -> row[:PULocationID] == PULocationID && row[:time_interval] == time_interval, df)
    
    # Calculate weights based on ride_percentage
    weights = filtered_df[:, :ride_percentage]
    # Randomly sample rows based on weights
    sampled_rows = sample(1:nrow(filtered_df), Weights(weights), 1)
    
    # Extract the sampled trip
    sampled_trip = filtered_df[sampled_rows, :]
    
    return sampled_trip
end

sample_trip (generic function with 1 method)

In [3]:
function expected_return(df, pickup,drop,current_time,rem_trips)
    gamma = 0.9
    pickup_df = DataFrame()

    #Fare calculation
    fare = 0 #initializing fare as 0 (stays as zero if drop = pickup)
    new_time = (current_time+1)%48 # initializing next timestep
    if pickup != drop
        pickup_df = filter(row -> row[:PULocationID] == pickup && row[:time_interval] == current_time && row[:DOLocationID] == drop, df)
        fare = pickup_df[1,"total_amount"]
        new_time = pickup_df[1,"next_time_interval"]
        println("Fare is ", fare)
        println("New time is ", new_time)
    end

    #Expected Returns Recursion
    drop_df = filter(row -> row[:PULocationID] == drop && row[:time_interval] == new_time, df) 
    
    if rem_trips == 0
        println("Terminate")
        return 0 #Termination condition: No trips remaining
    else
        weighted_sum = 0
        total_trips = sum(drop_df[!,"trips"])
        for dest in drop_df[!,"DOLocationID"] #list of destinations from the next pickup point
            p = filter(row -> row[:DOLocationID] == dest, drop_df)[1,"trips"] / total_trips
            # nt = get(first(eachrow(filter(row -> row.DOLocationID == dest, drop_df))), :next_time_interval, missing)
            println("recursing into case with drop =",drop," dest=", dest," new_time=",new_time," rem_trips=",rem_trips-1)
            weighted_sum += gamma * p * expected_return(df,drop,dest,new_time,rem_trips-1)
        end
        return fare+weighted_sum
    end
end
    

expected_return (generic function with 1 method)

In [4]:
function simulation(t,end_time,max_trips,start_location)
    revenue = 0
    rem_trips=max_trips
    while rem_trips>=0 && t<=end_time
        trip = sample_trip(trips, start_location, t);
        p = trip[1,"PULocationID"]
        d = trip[1,"DOLocationID"]
        t_trip = trip[1,"duration_interval"]

        
        model = Model(Gurobi.Optimizer);
    
        @variable(model, z>=0, Bin)
        @variable(model, Expected_Profit>=0)
        
        @objective(model, Max, Expected_Profit)
        
        @constraint(model, expected_profit, Expected_Profit == z*(expected_return(trips,p,d,t,rem_trips)) + (1-z)*(expected_return(trips,p,p,t,rem_trips)))
        @constraint(model, time_constraint, t + t_trip*z <= end_time)
        @constraint(model, num_trips_constraint, z<=max_trips-rem_trips)
        
        optimize!(model)
        if value(z)==1
            println("Driver should accept the trip")
            revenue+= trip[1,"total_amount"]
            println("Revenue generated:", revenue)
        else
            println("Driver should reject the trip")
        end
        
        t+= value(z) == 1 ? t_trip : 1;
        rem_trips -= value(z) == 1 ? 1 : 0;
        
        
    end
    return revenue
end

simulation (generic function with 1 method)

In [8]:
t = 20; #10am is the time driver receives first request
end_time = 40; #8pm is the time driver ends their shift
max_trips = 10; #driver is ready to accept a maximum of 10 trips per day
start_location = 4

# revenue = simulation(t,end_time,max_trips,start_location)

4

In [ ]:
#Linearizing
# CLusteirng
# Taking a subset of locations

In [ ]:
trip = sample_trip(trips, start_location, t);
p = trip[1,"PULocationID"]
d = trip[1,"DOLocationID"]
t_trip = trip[1,"duration_interval"]
rem_trips=max_trips

model = Model(Gurobi.Optimizer);

@variable(model, z>=0, Bin)
@variable(model, Expected_Profit>=0)

@objective(model, Max, Expected_Profit)

@constraint(model, expected_profit, Expected_Profit == z*(expected_return(trips,p,d,t,rem_trips)) + (1-z)*(expected_return(trips,p,p,t,rem_trips)))
@constraint(model, time_constraint, t + t_trip*z <= end_time)
@constraint(model, num_trips_constraint, z<=max_trips-rem_trips)

optimize!(model)
if value(z)==1
    println("Driver should accept the trip")
    revenue+= trip[1,"total_amount"]
    println("Revenue generated:", revenue)
else
    println("Driver should reject the trip")
end